In [ ]:
from keras.layers import LSTM
from keras.layers import Lambda
from keras.layers.merge import add
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dropout, Concatenate, ConvLSTM2D, Flatten, MaxPooling1D, Conv1D
from keras.layers import TimeDistributed
from keras.utils import to_categorical
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model
import csv
# convlstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
from matplotlib import pyplot
from itertools import product

In [ ]:
# click on "Upload file" and upload a zip of the dataset
# running this command will unzip it in the current dir
!unzip HARDataset.zip

In [ ]:
# helper function to save model results to csv files
def saveResults(name, fittingProcess, accuracy, aux_accuracy, loss, aux_loss, n):
    loss_history = fittingProcess.history['main_output_loss']
    acc_history = fittingProcess.history['main_output_acc']
    lstm_loss_history = fittingProcess.history['aux_output_loss']
    lstm_acc_history = fittingProcess.history['aux_output_acc']
    val_loss_history = fittingProcess.history['val_main_output_loss']
    val_acc_history = fittingProcess.history['val_main_output_acc']
    val_lstm_loss_history = fittingProcess.history['val_aux_output_loss']
    val_lstm_acc_history = fittingProcess.history['val_aux_output_acc']

    with open(name + str(n) + '.csv', "w") as outfile:
        outfile.write("loss,accuracy,val_loss,val_acc")
        outfile.write("\n")
        for ind in range(len(loss_history)):
            outfile.write(
                str(loss_history[ind]) + ',' + str(acc_history[ind]) + ',' + str(val_loss_history[ind]) + ',' + str(
                    val_acc_history[ind]))
            outfile.write("\n")

    with open(name + '-lstm' + str(n) + '.csv', "w") as outfile:
        outfile.write("lstm_loss,lstm_accuracy,val_lstm_loss,val_lstm_acc")
        outfile.write("\n")
        for ind in range(len(loss_history)):
            outfile.write(str(lstm_loss_history[ind]) + ',' + str(lstm_acc_history[ind]) + ',' + str(
                val_lstm_loss_history[ind]) + ',' + str(val_lstm_acc_history[ind]))
            outfile.write("\n")

    with open(name + '-modelevaluate' + str(n) + '.csv', "w") as outfile:
        outfile.write("loss,")
        outfile.write("accuracy")
        outfile.write("aux_loss,")
        outfile.write("aux_accuracy")
        outfile.write("\n")
        outfile.write(str(accuracy) + ',')
        outfile.write(str(loss) + ',')
        outfile.write(str(aux_accuracy))
        outfile.write(str(aux_loss))
        outfile.write("\n")

# helper function to assign hyperparameters
def unfold_general_hyperparameters(cfg):
    verbose = cfg.get('verbose') if ('verbose' in cfg) else 0
    epochs = cfg.get('epochs') if ('epochs' in cfg) else 25
    batch_size = cfg.get('batch_size') if ('batch_size' in cfg) else 64
    activation = cfg.get('activation') if ('activation' in cfg) else 'relu'
    # kernel_size_1D = cfg.get('kernel_size_1D') if ('kernel_size_1D' in cfg) else 3
    filters = cfg.get('filters') if ('filters' in cfg) else 64
    pool_size = cfg.get('pool_size') if ('pool_size' in cfg) else 2
    loss = cfg.get('loss') if ('loss' in cfg) else 'categorical_crossentropy'
    out_activation = cfg.get('out_activation') if ('out_activation' in cfg) else 'softmax'
    optimizer = cfg.get('optimizer') if ('optimizer' in cfg) else 'adam'
    dropout_rate = cfg.get('dropout_rate') if ('dropout_rate' in cfg) else 0.5

    return verbose, epochs, batch_size, activation, filters, pool_size, loss, out_activation, optimizer, dropout_rate

# helper function for PCA feature selection
def feature_selection(all_aux_trainX, all_aux_testX):
    data = np.concatenate((all_aux_trainX, all_aux_testX), axis=0)
    scaler = MinMaxScaler(feature_range=[0, 1])
    data_rescaled = scaler.fit_transform(data)
    pca = PCA(n_components=175)
    dataset = pca.fit_transform(data_rescaled)
    aux_trainX = dataset[0:all_aux_trainX.shape[0]][:]
    aux_testX = dataset[all_aux_trainX.shape[0]:][:]
    return aux_trainX, aux_testX

# residual lstm layer generator
def residual_lstm_layers(input, rnn_width, rnn_depth, rnn_dropout):
    x = input
    for i in range(rnn_depth):
        return_sequences = i < rnn_depth - 1
        # if the return_sequences is true, which means that this LSTM layer will output 3D instead of 2D(By default LSTM output 2D(the last time step of sequence)).
        # have the LSTM output a value for each time step in the input data.
        x_rnn = LSTM(rnn_width, recurrent_dropout=rnn_dropout, dropout=rnn_dropout, return_sequences=return_sequences)(
            x)
        if return_sequences:

            if i > 0 or input.shape[-1] == rnn_width:
                x = add([x, x_rnn])
            else:

                x = x_rnn
        else:
            # Last layer does not return sequences, just the last element
            # so we select only the last element of the previous output.
            def slice_last(x):
                return x[..., -1, :]

            x = add([Lambda(slice_last)(x), x_rnn])
            # x = TimeDistributed(Dense(6, activation='softmax'))(x)
    return x

In [ ]:
# fit and evaluate a multi-input/multi-output ConvLSTM model
def evaluate_convlstm_multi_model(trainX, trainy, testX, testy, aux_trainX, aux_trainy, aux_testX, aux_testy, cfg, n):
    ## datastuff
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    n_aux_features = aux_trainX.shape[1]
    # reshape into subsequences (samples, time steps, rows, cols, channels)
    n_steps, n_length = 4, 32
    trainX = trainX.reshape((trainX.shape[0], n_steps, 1, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, 1, n_length, n_features))

    ## parameterstuff
    verbose, epochs, batch_size, activation, \
    filters, pool_size, loss, out_activation, \
    optimizer, dropout_rate = unfold_general_hyperparameters(cfg)
    kernel_size_2D = cfg.get('kernel_size_2D') if ('kernel_size_2D' in cfg) else (1, 3)

    ## modelstuff
    main_input = Input(shape=(n_steps, 1, n_length, n_features), dtype='float32', name='main_input')
    lstm_out = ConvLSTM2D(filters=filters, kernel_size=kernel_size_2D, activation=activation)(main_input)
    x = Dropout(rate=dropout_rate)(lstm_out)
    x = Flatten()(x)
    x = Dense(100, activation=activation)(x)
    # auxiliary output of CNNLSTM part
    auxiliary_output = Dense(n_outputs, activation=out_activation, name='aux_output')(x)

    # flatten output
    lstm_out_flat = Flatten()(lstm_out)
    auxiliary_input = Input(shape=(n_aux_features,), name='aux_input')
    # combine inputs
    x = Concatenate()([lstm_out_flat, auxiliary_input])
    # rest of the network
    x = Dense(128, activation=activation)(x)
    x = Dense(64, activation=activation)(x)
    x = Dense(32, activation=activation)(x)
    # final output
    main_output = Dense(n_outputs, activation=out_activation, name='main_output')(x)
    model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output, auxiliary_output])
    model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

    ## train/test
    history = model.fit(x=[trainX, aux_trainX], y=[trainy, aux_trainy], epochs=epochs, batch_size=batch_size,
                        verbose=verbose, validation_data=([testX, aux_testX], [testy, aux_testy]))
    # print(history.history)
    # print(model.metrics_names)
    # print(model.summary())
    _, loss, aux_loss, accuracy, aux_acc = model.evaluate(x=[testX, aux_testX], y=[testy, aux_testy],
                                                          batch_size=batch_size, verbose=1)
    saveResults("convLstmMulti", history, accuracy, aux_acc, loss, aux_loss, n)

    return accuracy, aux_acc


# fit and evaluate a multi-input/multi-output CNN-LSTM model
def evaluate_cnnlstm_multi_model(trainX, trainy, testX, testy, aux_trainX, aux_trainy, aux_testX, aux_testy, cfg, n):
    ## data stuff
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    # reshape data into time steps of sub-sequences
    n_steps, n_length = 4, 32
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))


    ## parameterstuff
    verbose, epochs, batch_size, activation,\
    filters, pool_size, loss, out_activation, \
    optimizer, dropout_rate = unfold_general_hyperparameters(cfg)
    kernel_size_1D = cfg.get('kernel_size_1D') if ('kernel_size_1D' in cfg) else 3

    ## define model
    main_input = Input(shape=(None, n_length, n_features), dtype='float32', name='main_input')
    x = TimeDistributed(Conv1D(filters=filters, kernel_size=kernel_size_1D, activation='relu'))(main_input)
    x = TimeDistributed(Conv1D(filters=filters, kernel_size=kernel_size_1D, activation='relu'))(x)
    x = TimeDistributed(Dropout(dropout_rate))(x)
    x = TimeDistributed(MaxPooling1D(pool_size=pool_size))(x)
    x = TimeDistributed(Flatten())(x)
    lstm_out = LSTM(units=100)(x)
    x = Dropout(rate=dropout_rate)(lstm_out)
    x = Dense(100, activation=activation)(x)
    auxiliary_output = Dense(n_outputs, activation=out_activation, name='aux_output')(x)
    num_features = aux_trainX.shape[1]
    auxiliary_input = Input(shape=(num_features,), name='aux_input')
    # combine inputs
    x = Concatenate()([lstm_out, auxiliary_input])
    # rest of the network
    x = Dense(128, activation=activation)(x)
    x = Dense(64, activation=activation)(x)
    x = Dense(32, activation=activation)(x)
    # final output
    main_output = Dense(n_outputs, activation=out_activation, name='main_output')(x)
    model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output, auxiliary_output])
    model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

    history = model.fit(x=[trainX, aux_trainX], y=[trainy, aux_trainy], epochs=epochs, batch_size=batch_size,
                        verbose=verbose, validation_data=([testX, aux_testX], [testy, aux_testy]))
    _, loss, aux_loss, accuracy, aux_acc = model.evaluate(x=[testX, aux_testX], y=[testy, aux_testy],
                                                          batch_size=batch_size, verbose=1)

    saveResults("cnnLstmMulti", history, accuracy, aux_acc, loss, aux_loss, n)
    return accuracy, aux_acc


# fit and evaluate a multi-input/multi-output residual LSTM model
def evaluate_res_lstm_multi_model(trainX, trainy, testX, testy, aux_trainX, aux_trainy, aux_testX, aux_testy, cfg, n):
    verbose, epochs, batch_size = 0, 25, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    # define model
    main_input_res = Input(shape=(n_timesteps, n_features), name='residual_lstm_input')
    lstm_out = residual_lstm_layers(main_input_res, rnn_width=9, rnn_depth=4, rnn_dropout=0.2)
    dense_out=Dense(100, activation='relu')(lstm_out)
    auxiliary_output=Dense(n_outputs, activation='softmax', name='aux_output')(dense_out)
    num_features = aux_trainX.shape[1]
    auxiliary_input = Input(shape=(num_features,), name='aux_input')
    # combine inputs
    x = Concatenate()([lstm_out, auxiliary_input])
    # rest of the network
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    # final output
    main_output = Dense(n_outputs, activation='softmax', name='main_output')(x)
    model = Model(inputs=[main_input_res, auxiliary_input], outputs=[main_output, auxiliary_output])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    history = model.fit(x=[trainX, aux_trainX], y=[trainy, aux_trainy], epochs=epochs, batch_size=batch_size,
                        verbose=verbose, validation_data=([testX, aux_testX], [testy, aux_testy]))
    _, loss, aux_loss, accuracy, aux_acc = model.evaluate(x=[testX, aux_testX], y=[testy, aux_testy],
                                                          batch_size=batch_size, verbose=1)
    saveResults("resLstmMulti", history, accuracy, aux_acc, loss, aux_loss, n)
    return accuracy, aux_acc


# fit and evaluate a multi-input/multi-output stacked LSTM model
def evaluate_stacked_lstm_multi_model(trainX, trainy, testX, testy, aux_trainX, aux_trainy, aux_testX, aux_testy, cfg, n):
    verbose, epochs, batch_size = 0, 25, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    main_input_stacked = Input(shape=(n_timesteps, n_features), name='stacked_lstm_input')
    lstm_out0=LSTM(9, activation='tanh',recurrent_dropout=0.2, dropout=0.2, return_sequences=True)(main_input_stacked)
    lstm_out1=LSTM(9, activation='tanh',recurrent_dropout=0.2, dropout=0.2, return_sequences=True)(lstm_out0)
    lstm_out2=LSTM(9, activation='tanh',recurrent_dropout=0.2, dropout=0.2, return_sequences=True)(lstm_out1)
    lstm_out3=LSTM(9, activation='tanh',recurrent_dropout=0.2, dropout=0.2, return_sequences=False)(lstm_out2)
    #lstm_out4=LSTM(9, activation='tanh',recurrent_dropout=0.2, dropout=0.2, return_sequences=False)(lstm_out3)
    Dense_out = Dense(100, activation='relu')(lstm_out3)
    auxiliary_output = Dense(n_outputs, activation='softmax', name='aux_output')(Dense_out)
    num_features = aux_trainX.shape[1]
    auxiliary_input = Input(shape=(num_features,), name='aux_input')
    # combine inputs
    x = Concatenate()([lstm_out3, auxiliary_input])
    # rest of the network
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    # final output
    main_output = Dense(n_outputs, activation='softmax', name='main_output')(x)
    model = Model(inputs=[main_input_stacked, auxiliary_input], outputs=[main_output, auxiliary_output])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    history = model.fit(x=[trainX, aux_trainX], y=[trainy, aux_trainy], epochs=epochs, batch_size=batch_size,
                        verbose=verbose, validation_data=([testX, aux_testX], [testy, aux_testy]))
    _, loss, aux_loss, accuracy, aux_acc = model.evaluate(x=[testX, aux_testX], y=[testy, aux_testy],
                                                          batch_size=batch_size, verbose=1)
    saveResults("stackedLstmMulti", history, accuracy, aux_acc, loss, aux_loss, n)
    return accuracy, aux_acc


# fit and evaluate ConvLSTM a model
def evaluate_convlst_model(trainX, trainy, testX, testy, cfg):
    # define model
    verbose, epochs, batch_size = 0, 25, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    # reshape into subsequences (samples, time steps, rows, cols, channels)
    n_steps, n_length = 4, 32
    trainX = trainX.reshape((trainX.shape[0], n_steps, 1, n_length, n_features))
    # trainX = trainX
    # trainy = trainy
    testX = testX.reshape((testX.shape[0], n_steps, 1, n_length, n_features))
    # define model
    model = Sequential()
    model.add(
        ConvLSTM2D(filters=64, kernel_size=(1, 3), activation='relu', input_shape=(n_steps, 1, n_length, n_features)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy


# fit and evaluate a CNN-LSTM model
def evaluate_cnnlstm_model(trainX, trainy, testX, testy, cfg):
    # define model
    verbose, epochs, batch_size = 0, 25, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    # reshape data into time steps of sub-sequences
    n_steps, n_length = 4, 32
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    # define model
    model = Sequential()
    model.add(
        TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None, n_length, n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy


# fit and evaluate a residual LSTM model
def evaluate_res_lstm_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 0, 25, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    input_res = Input(shape=(n_timesteps, n_features), name='residual_lstm_input')
    lstm_out = residual_lstm_layers(input_res, rnn_width=9, rnn_depth=8, rnn_dropout=0.2)
    Dense_out=Dense(100, activation='relu')(lstm_out)
    output=Dense(n_outputs,activation='softmax')(Dense_out)
    model = Model(inputs=input_res, outputs=output)
    # model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    # fit the network
    model.fit(x=trainX, y=trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    _, accuracy = model.evaluate(x=testX, y=testy, batch_size=batch_size, verbose=verbose)
    return accuracy


# fit and evaluate a stacked LSTM model
def evaluate_stacked_lstm_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 0, 25, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    input_stacked = Input(shape=(n_timesteps, n_features), name='stacked_lstm_input')
    lstm_out0=LSTM(9, activation='tanh',recurrent_dropout=0.2, dropout=0.2, return_sequences=True)(input_stacked)
    lstm_out1=LSTM(9, activation='tanh',recurrent_dropout=0.2, dropout=0.2, return_sequences=True)(lstm_out0)
    lstm_out2=LSTM(9, activation='tanh',recurrent_dropout=0.2, dropout=0.2, return_sequences=True)(lstm_out1)
    lstm_out3=LSTM(9, activation='tanh',recurrent_dropout=0.2, dropout=0.2, return_sequences=True)(lstm_out2)
    lstm_out4=LSTM(9, activation='tanh',recurrent_dropout=0.2, dropout=0.2, return_sequences=False)(lstm_out3)
    Dense_out = Dense(100, activation='relu')(lstm_out4)
    output = Dense(n_outputs, activation='softmax')(Dense_out)
    model = Model(inputs=input_stacked, outputs=output)
    # model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    # fit the network
    model.fit(x=trainX, y=trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    _, accuracy = model.evaluate(x=testX, y=testy, batch_size=batch_size, verbose=verbose)
    return accuracy

In [ ]:

# how to use:

# in run experiment: set grid to true or not, set amount of repeats (cross validation)
# in run experiment: set the evaluate function, choose from the allModels file
# in giveParameters: set values for the hyper-parameters to be tested

# the current implementation will poop out (value of repeat) amount of csv's with results during each epoch,
# it will override these files per configuration.

# if grid is enabled it will print the best configuration at the end, use this anew with only those parameters in giveParameters to get the csv's of the best config

def giveParameters():
    # learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
    verbose = [1]
    batch_size = [64, 128]
    optimizer = ['adam', 'sgd']
    epochs = [50]
    activation = ['relu', 'sigmoid', 'tanh']
    kernel_size_2D = [(1, 3)]
    kernel_size_1D = [3]
    filters = [64, 128]
    pool_size = [2]
    loss = ['categorical_crossentropy']
    out_activation = ['softmax']
    dropout_rate = [0, 0.25, 0.5]
    return dict(verbose=verbose, epochs=epochs, batch_size=batch_size, activation=activation,
                kernel_size_2D=kernel_size_2D, kernel_size_1D=kernel_size_1D, filters=filters, pool_size=pool_size,
                loss=loss, out_activation=out_activation, optimizer=optimizer, dropout_rate=dropout_rate)


# run an experiment
def run_experiment(repeats=10):
    # load data
    trainX, trainy, testX, testy, aux_trainX, aux_trainy, aux_testX, aux_testy = load_dataset()
    grid = True

    cfg_list = defineConfigurations()  # list with all possible configurations

    gridresults = list()

    for cfg in cfg_list:
        scores = list()
        for r in range(repeats):
            score, aux_score = evaluate_stacked_lstm_multi_model(trainX, trainy, testX, testy, aux_trainX, aux_trainy,
                                                            aux_testX, aux_testy, cfg,
                                                            r)  # change if you want to run another model
            score = score * 100.0
            aux_score = aux_score * 100.00  # also remove everything regarding aux_score if a different model is used
            print('>#%d: LSTM = %.3f and Multi = %.3f' % (r + 1, score, aux_score))
            scores.append(score)
        gridresults.append((cfg, scores))

    if grid:
        summarize_gridresults(gridresults)
    printBestGrid(gridresults)


########################### grid functions

def summarize_gridresults(gridresults):
    for x in gridresults:
        print(x[0])  # prints the config
        summarize_results(x[1])  # prints the scores


def defineConfigurations():
    cfgs = list()
    parameters = giveParameters()
    return list((dict(zip(parameters, x)) for x in product(*parameters.values())))


def printBestGrid(gridresults):
    best = gridresults[0]
    for x in gridresults:
        if mean(x[1]) > mean(best[1]):
            best = x
    print("best result:")
    print(best[0])
    summarize_results(best[1])


########################### preparing the data


# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values


# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    loaded = dstack(loaded)
    return loaded


# load a dataset group, such as train or test
def load_timeseries_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_' + group + '.txt', 'total_acc_y_' + group + '.txt', 'total_acc_z_' + group + '.txt']
    # body acceleration
    filenames += ['body_acc_x_' + group + '.txt', 'body_acc_y_' + group + '.txt', 'body_acc_z_' + group + '.txt']
    # body gyroscope
    filenames += ['body_gyro_x_' + group + '.txt', 'body_gyro_y_' + group + '.txt', 'body_gyro_z_' + group + '.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_' + group + '.txt')
    return X, y


def load_const_dataset_group(group, prefix=''):
    X = load_file(prefix + group + '/X_' + group + '.txt')
    y = load_file(prefix + group + '/y_' + group + '.txt')
    return X, y




# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_timeseries_dataset_group('train', prefix + 'HARDataset/')
    all_aux_trainX, aux_trainy = load_const_dataset_group('train', prefix + 'HARDataset/')
    print(">> Time series data shape: {0} , {1}".format(trainX.shape, trainy.shape))
    print(">> Constant data shape: {0} , {1}".format(all_aux_trainX.shape, aux_trainy.shape))
    # load all test
    testX, testy = load_timeseries_dataset_group('test', prefix + 'HARDataset/')
    all_aux_testX, aux_testy = load_const_dataset_group('test', prefix + 'HARDataset/')
    print(">> Time series data shape: {0} , {1}".format(testX.shape, testy.shape))
    print(">> Constant data shape: {0} , {1}".format(all_aux_testX.shape, aux_testy.shape))
    # feature selection on constant features
    aux_trainX, aux_testX = feature_selection(all_aux_trainX, all_aux_testX)
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    aux_trainy = aux_trainy - 1
    aux_testy = aux_testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    aux_trainy = to_categorical(aux_trainy)
    aux_testy = to_categorical(aux_testy)
    print(">> Final shapes of time series dataset: {0}, {1}, {2}, {3}".format(trainX.shape, trainy.shape, testX.shape,
                                                                              testy.shape))
    print(">> Final shapes of constant dataset: {0}, {1}, {2}, {3}".format(aux_trainX.shape, aux_trainy.shape,
                                                                           aux_testX.shape, aux_testy.shape))
    return trainX, trainy, testX, testy, aux_trainX, aux_trainy, aux_testX, aux_testy


########################### summarize scores

def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [ ]:
########################### run the experiment

run_experiment()